In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
spreadsheet = gc.open_by_key("17t51PY6ctD29Z6evAKr64mrRWc64VDPZ9HvtIzJSMzA")
sheets = list(filter(lambda sheet: len(sheet.title) == 4 and sheet.title != "2021", spreadsheet.worksheets()))
print(sheets)

[<Worksheet '2020' id:1819157839>, <Worksheet '2019' id:0>, <Worksheet '2018' id:700803315>, <Worksheet '2017' id:1016706833>, <Worksheet '2016' id:283370662>, <Worksheet '2015' id:1277267242>]


In [2]:
import pandas as pd
COUNTRIES = []
years = {}

for sheet in sheets:
  year = int(sheet.title)
  data = list(sheet.get("A3:AF200"))
  rows = [[None for x in range(len(data[0]))] for y in range(len(data))]
  index = []
  print(year)

  for y, col in enumerate(data):
    for x, cell in enumerate(col):
      if x == 0:
        index.append(cell)
      
      rows[y][x] = cell

      if x > 0 and y > 0:
        rows[y][x] = None if str.strip(cell) == "" else int(cell)
  
  COUNTRIES = rows[0]
  df = pd.DataFrame(rows[1:], columns=COUNTRIES).fillna(0)

  # filter by A and D assets
  df = df[df.Type.str.startswith("A") | df.Type.str.startswith("D")]


  # remove prefixes
  df = df.replace({r'^[AD] - (.*)$': r'\1'}, regex=True)

  # sum up A and D
  df = df.groupby('Type').agg(sum)

  years[year] = df

YEARS = dict.keys(years)
COUNTRIES.remove("Type")

2020
2019
2018
2017
2016
2015


In [3]:
years[2020].head(200)

,AT,BE,BG,CH,CY,CZ,DE,DK,EE,GR,...,NL,NO,PL,PT,RO,SE,SI,SK,GB,FX
Type,,,,,,,,,,,,,,,,,,,,,
CO2 Detector,0,0,0,0,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,26.0
Coastal Patrol Boat (CPB),0,0,7,0,0,0,2,1,4,19,...,1,0,0,11,12,1,0,0,0,0.0
Coastal Patrol Vessel (CPV),0,0,8,0,0,0,0,0,0,0,...,0,0,0,2,12,0,0,0,0,0.0
Deportation Officers,101,4,0,0,1,0,36,4,5,0,...,38,4,4,10,5,4,0,14,0,0.0
Fixed Wing Aircraft (FWA),0,0,0,0,0,0,0,3,0,1,...,1,0,0,2,0,1,0,1,0,5.0
Heartbeat Detector,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10.0
Helicopter,0,0,0,0,0,0,2,1,0,3,...,0,0,1,0,2,0,0,0,0,0.0
Mobile Laboratory,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
Offshore Patrol Vessel (OPV),0,0,0,0,0,0,0,0,0,11,...,0,0,0,0,1,0,0,0,0,0.0


In [4]:
import altair as alt

# get all officer deployments over the years
data = {
    "officers": [df[df.index.str.contains('Officer')].sum(axis=1).sum() for df in dict.values(years)],
    "detectors": [df[df.index.str.contains('Detector')].sum(axis=1).sum() for df in dict.values(years)]
}
df = pd.DataFrame(data, index=YEARS)
base = alt.Chart(df.reset_index()).encode(
    alt.X('index',
        title='year',
        axis=alt.Axis(format="", tickCount=len(YEARS)),
        scale=alt.Scale(domain=(min(YEARS), max(YEARS)))
    )
)

officers = base.mark_line().encode(y='officers')
#detectors= base.mark_line(stroke='#57A44C').encode(y='detectors')

alt.layer(officers).resolve_scale(y='independent')

alt.LayerChart(...)

In [5]:
import json
json.dumps({k: df.to_dict() for k, df in dict.items(years)})

'{"2020": {"AT": {"CO2 Detector": 0, "Coastal Patrol Boat (CPB)": 0, "Coastal Patrol Vessel (CPV)": 0, "Deportation Officers": 101, "Fixed Wing Aircraft (FWA)": 0, "Heartbeat Detector": 0, "Helicopter": 0, "Mobile Laboratory": 0, "Offshore Patrol Vessel (OPV)": 0, "Operation Officers": 239, "Other": 0, "Other Officers": 0, "Patrol Car": 44, "Thermo-Vision Vehicle (TVV)": 25, "Transportation Vehicle/Canine Team Vehicle": 0}, "BE": {"CO2 Detector": 0, "Coastal Patrol Boat (CPB)": 0, "Coastal Patrol Vessel (CPV)": 0, "Deportation Officers": 4, "Fixed Wing Aircraft (FWA)": 0, "Heartbeat Detector": 0, "Helicopter": 0, "Mobile Laboratory": 0, "Offshore Patrol Vessel (OPV)": 0, "Operation Officers": 40, "Other": 0, "Other Officers": 0, "Patrol Car": 0, "Thermo-Vision Vehicle (TVV)": 0, "Transportation Vehicle/Canine Team Vehicle": 0}, "BG": {"CO2 Detector": 0, "Coastal Patrol Boat (CPB)": 7, "Coastal Patrol Vessel (CPV)": 8, "Deportation Officers": 0, "Fixed Wing Aircraft (FWA)": 0, "Heartbea

In [6]:
# asset groups

assets = [
          ('officers', ['Deportation Officers', 'Operation Officers', 'Other Officers']),
          ('detectors', ['CO2 Detector', 'Heartbeat Detector']),
          ('dogTeam', ['Dog Team']),
          ('vessels', ['Coastal Patrol Boat (CPB)', 'Offshore Patrol Vessel (OPV)', 'Coastal Patrol Boat (CPB)']),
          ('aircrafts', ['Helicopter', 'Fixed Wing Aircraft (FWA)']),
          ('patrolCar', ['Patrol Car'])
]

In [7]:
# per-country, per-year data export

import json

output = {}
for country in COUNTRIES:
  output[country] = []

  for (year, df) in dict.items(years):
    obj = { "year": year }

    for (asset, groups) in assets:
      obj[asset] = 0

      for item in groups:
        obj[asset] += int(df[country].get(item) or 0)
    
    output[country].append(obj)

json.dumps(output)

'{"AT": [{"year": 2020, "officers": 340, "detectors": 0, "dogTeam": 0, "vessels": 0, "aircrafts": 0, "patrolCar": 44}, {"year": 2019, "officers": 376, "detectors": 0, "dogTeam": 0, "vessels": 0, "aircrafts": 0, "patrolCar": 7}, {"year": 2018, "officers": 338, "detectors": 0, "dogTeam": 0, "vessels": 0, "aircrafts": 0, "patrolCar": 13}, {"year": 2017, "officers": 393, "detectors": 0, "dogTeam": 2, "vessels": 0, "aircrafts": 0, "patrolCar": 57}, {"year": 2016, "officers": 0, "detectors": 0, "dogTeam": 11, "vessels": 0, "aircrafts": 0, "patrolCar": 48}, {"year": 2015, "officers": 0, "detectors": 0, "dogTeam": 2, "vessels": 0, "aircrafts": 0, "patrolCar": 4}], "BE": [{"year": 2020, "officers": 44, "detectors": 0, "dogTeam": 0, "vessels": 0, "aircrafts": 0, "patrolCar": 0}, {"year": 2019, "officers": 96, "detectors": 0, "dogTeam": 0, "vessels": 0, "aircrafts": 0, "patrolCar": 0}, {"year": 2018, "officers": 64, "detectors": 0, "dogTeam": 0, "vessels": 0, "aircrafts": 0, "patrolCar": 0}, {"ye